In [ ]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization,Activation
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


import warnings
warnings.filterwarnings("ignore")

print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.3.1


In [ ]:
import pickle
def unpickle(file):
    with open(file, 'rb') as fo: #read in binary mode rb
        dict = pickle.load(fo,  encoding='latin1')
    return dict

def reshape(dataset):
    return np.reshape(dataset, (-1, 3, 32, 32)).transpose((0, 2, 3, 1))

In [ ]:
def load_dataset():
    train  = unpickle('./cifar-100-python/train')
    test = unpickle('./cifar-100-python/test')
    meta = unpickle('./cifar-100-python/meta')
    #print(train.keys())
    #print(test.keys())
    #print(meta.keys())
    trainX = train['data']
    trainY = np.array(train['fine_labels'])
    super_trainY = np.array(train['coarse_labels'])

    testX = test['data']
    testY = np.array(test['fine_labels'])
    super_testY = np.array(test['coarse_labels'])

    label_names = meta['fine_label_names']
    super_label_names = meta['coarse_label_names']
    return trainX, trainY, testX, testY

In [ ]:
def process_data(trainX, trainY, testX, testY):
    # Pre-Process Data
    trainX = trainX.astype('float32')
    trainX = trainX / 255.0
    trainX = reshape(trainX)
    
    testX = testX.astype('float32')
    testX = testX / 255.0
    testX = reshape(testX)

    trainY = to_categorical(trainY)
    testY = to_categorical(testY)

    #super_trainY = to_categorical(super_trainY, num_classes = 100)
    #super_testY = to_categorical(super_testY, num_classes = 100)

    
    return  trainX, trainY, testX, testY

In [ ]:
x_train, y_train, x_test, y_test = load_dataset()
x_train, y_train, x_test, y_test = process_data(x_train, y_train, x_test, y_test)   
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 100)
Test: X=(10000, 32, 32, 3), y=(10000, 100)


In [ ]:
input_shape=(32,32,3)
model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=input_shape) 

x=model_d.output

x= BatchNormalization()(x)
x= Activation('relu')(x)
x= GlobalAveragePooling2D()(x)

# x= GlobalAveragePooling2D()(x)
# x= BatchNormalization()(x)
# x= Dropout(0.5)(x)
# x= Dense(1024,activation='relu')(x) 
# x= Dense(512,activation='relu')(x) 
# x= BatchNormalization()(x)
# x= Dropout(0.5)(x)

preds=Dense(100,activation='softmax')(x) #FC-layer

model=Model(inputs=model_d.input, outputs=preds)
model.summary()

#tf.keras.utils.plot_model( model , show_shapes=True )

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

batch_size = 128
epochs = 60

#(xtrain,xtest,ytrain,ytest)=train_test_split(x_train,y_train,test_size=0.4,random_state=42)
#print(xtrain.shape, xtest.shape)

history = model.fit( x_train , y_train , epochs=epochs , batch_size=batch_size )

print('Saving Model...')

#model.save("m_densenet121.h5")

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
_______________________________________________________________________________________

KeyboardInterrupt: 

## Teset with Image generator

In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen.fit(xtrain)
# Fits-the-model
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=128),
               steps_per_epoch=xtrain.shape[0] // 128,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

In [ ]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

### heat map

In [ ]:
Ypred = model.predict(xtest)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(ytest, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)